In [ ]:
# importing the required libraries
import pandas as pd
import numpy as np
import textwrap
import os
import json
from tqdm import tqdm
import nltk
import regex as re


In [ ]:
os.getcwd()

In [ ]:
# enter the base_path
base_path = ""
store_path = base_path + "/" +"10_K_word_count/"
files_path = base_path + "/" +"10_K/"

In [ ]:
# getting file names
files = os.listdir(files_path)
len(files)

174

In [ ]:
# testing one file
fp = open(files_path+files[0], 'r')
df = json.load(fp)

In [ ]:
# get stopwords list from nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [ ]:
text = df['item_7'] +df['item_7A']

def give_word_count_dict(text):
    # clean text using regex
    text = re.sub(r'[^\w\s]','',text)
    # remove stop words
    text = ' '.join([word for word in text.split() if word not in stopwords])

    # split text into words
    words = text.split()
    # create a dictionary with word as key and its count as value
    word_count_dict = {}
    for word in words:
        if word in word_count_dict:
            word_count_dict[word] += 1
        else:
            word_count_dict[word] = 1
    return word_count_dict


word_count_dict = give_word_count_dict(text)

In [ ]:
def give_bigram_and_trigram_dict(text):
    # clean text using regex
    text = re.sub(r'[^\w\s]','',text)
    # remove stop words
    text = ' '.join([word for word in text.split() if word not in stopwords])
    # now get ngram counts in the text using nltk
    bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(text.split())
    trigram_finder = nltk.collocations.TrigramCollocationFinder.from_words(text.split())
    # get bigram and trigram counts
    bigram_counts = bigram_finder.ngram_fd.items()
    trigram_counts = trigram_finder.ngram_fd.items()
    # create a dictionary with bigram and trigram as key and its count as value
    bigram_dict = {}
    trigram_dict = {}
    for bigram, count in bigram_counts:
        bigram_dict[bigram[0]+ " " +bigram[1]] = count
    for trigram, count in trigram_counts:
        trigram_dict[trigram[0]+ " " +trigram[1]] = count
    return bigram_dict, trigram_dict

In [ ]:
# get the bigram and trigram counts
bigram_dict, trigram_dict = give_bigram_and_trigram_dict(text)

# sort both the dicts on th basis of their values
bigram_dict = dict(sorted(bigram_dict.items(), key=lambda x: x[1], reverse=True))
trigram_dict = dict(sorted(trigram_dict.items(), key=lambda x: x[1], reverse=True))


# save both of these in a csv file
d2 = pd.DataFrame(bigram_dict.items(),columns=['bigram', 'bigram_count'])
d3 = pd.DataFrame(trigram_dict.items(), columns=['trigram', 'trigram_count'])
d2.head()

,bigram,bigram_count
0,December 31,78
1,ended December,61
2,The Company,58
3,year ended,53
4,The Companys,37


In [ ]:
d3.head()

,trigram,trigram_count
0,ended December,61
1,year ended,53
2,compared year,12
3,foreign exchange,10
4,million year,10


In [ ]:
# sort the word count dictionary in descending order
sorted_word_count_dict = sorted(word_count_dict.items(), key=lambda x: x[1], reverse=True)
# save sorted_word_count_dict to a csv file
d = pd.DataFrame(sorted_word_count_dict, columns=['word', 'count'])
d.head()

,word,count
0,The,181
1,million,128
2,Companys,124
3,Company,121
4,year,95


In [ ]:
# provided a text input, this function gives all three word count, bigram_count and trigram_count
def give_all_three_occurences(text):
    
    # clean text using regex
    text = re.sub(r'[^\w\s]','',text)
    # remove stop words
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # now get ngram counts in the text using nltk
    bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(text.split())
    trigram_finder = nltk.collocations.TrigramCollocationFinder.from_words(text.split())
    # get bigram and trigram counts
    bigram_counts = bigram_finder.ngram_fd.items()
    trigram_counts = trigram_finder.ngram_fd.items()
    # create a dictionary with bigram and trigram as key and its count as value
    bigram_dict = {}
    trigram_dict = {}
    for bigram, count in bigram_counts:
        bigram_dict[bigram[0]+ " " +bigram[1]] = count
    for trigram, count in trigram_counts:
        trigram_dict[trigram[0]+ " " +trigram[1]] = count

    # split text into words
    words = text.split()
    # create a dictionary with word as key and its count as value
    word_count_dict = {}
    for word in words:
        if word in word_count_dict:
            word_count_dict[word] += 1
        else:
            word_count_dict[word] = 1

    # sort both the dicts on th basis of their values
    bigram_dict = dict(sorted(bigram_dict.items(), key=lambda x: x[1], reverse=True))
    trigram_dict = dict(sorted(trigram_dict.items(),key=lambda x: x[1], reverse=True))

    # sort the word count dictionary in descending order
    sorted_word_count_dict = sorted(word_count_dict.items(), key=lambda x: x[1], reverse=True)
    
    d = pd.DataFrame(sorted_word_count_dict, columns=['word', 'count'])
    d2 = pd.DataFrame(bigram_dict.items(),columns=['bigram', 'bigram_count'])
    d3 = pd.DataFrame(trigram_dict.items(), columns=['trigram', 'trigram_count'])

    return d,d2,d3

In [ ]:
# this list to check some of badly scraped files for correction
files_that_dont_have_item_7A = []
for i in tqdm(files,position=0, leave = True):
    try:
        fp = open(files_path+i, 'r')
        df = json.load(fp)
        fp.close()
        text = df['item_7'] + df['item_7A']
        d,d2,d3 = give_all_three_occurences(text)
        d.to_csv(store_path+'word_count_'+i[:-5]+'.csv')
        d2.to_csv(store_path+'bigram_count_'+i[:-5]+'.csv')
        d3.to_csv(store_path+'trigram_count_'+i[:-5]+'.csv')
    except:
        files_that_dont_have_item_7A.append(i)


100%|██████████| 174/174 [00:31<00:00,  5.51it/s]


In [ ]:
files_that_dont_have_item_7A

['10K_1342936_20151231_0001342936-16-000057.json',
 '10K_1342936_20161231_0001342936-17-000016.json',
 '10K_1342936_20171231_0001342936-18-000018.json',
 '10K_1342936_20181231_0001342936-19-000004.json',
 '10K_1342936_20191231_0001342936-20-000011.json',
 '10K_1342936_20201231_0001376474-21-000115.json',
 '10K_869531_20131231_0001127855-17-000067.json',
 '10K_869531_20141231_0001127855-17-000191.json',
 '10K_869531_20151231_0001127855-17-000195.json',
 '10K_869531_20161231_0001127855-17-000252.json',
 '10K_869531_20171231_0001127855-18-000049.json',
 '10K_869531_20181231_0001654954-19-003723.json']